# Ocean BRDF Benchmark

In [41]:
import json
import os
import drjit as dr
import mitsuba as mi
import numpy as np
import matplotlib.pyplot as plt

## Loading Data

First, we load the data from the data files. This JSON file is a dictionary which contains:

* (The wavelength at which the data was generated)
* (The wind speed at which the data was generated)
* Oceanic properties such as salinity, chlorinity and pigmentation
* The data itself, consisting of incoming directions, for which multiple outgoing directions have been evaluated

In [42]:
wind_speed_benchmark = 10
wavelength_benchmark = 2.2

def load_data():
    with open(f'data/data_{wind_speed_benchmark}ms_{wavelength_benchmark}.json') as file:
        data = json.load(file)
        return data

In [43]:
benchmark_data = load_data()

# Obtain the other parameters from the data
solar_azimuth = benchmark_data['solar_azimuth']
view_azimuth = benchmark_data['view_azimuth']
wind_direction = benchmark_data['wind_direction']
salinity = benchmark_data['salinity']
chlorinity = benchmark_data['chlorinity']
pigmentation = benchmark_data['pigmentation']

# Solar Data
solar_data = benchmark_data['data']

Next, we can construct each benchmark entry. The solar data contains an array of different solar zeniths. For each of these zeniths, many outgoing directions have been sampled. One sample from this dataset is **a complete observation for a given solar zenith**.

In [44]:
for key, entry in solar_data.items():
    solar_zenith = float(key)

    # Extract the data
    for key, item in entry.items():
        outgoing_zenith = item['outgoing_zenith']
        foam = item['foam']
        glint = item['glint']
        water = item['water']
        total = item['total']
        foam_percent = item['foam_percent']
        glint_percent = item['glint_percent']
        water_percent = item['water_percent']


0.0
0.03530325077897621
0.07060650155795242
0.10590975233692863
0.14121300311590484
0.17651625389488107
0.21181950467385727
0.24712275545283346
0.2824260062318097
0.3177292570107859
0.35303250778976214
0.3883357585687383
0.42363900934771453
0.45894226012669076
0.49424551090566693
0.5295487616846432
0.5648520124636194
0.6001552632425956
0.6354585140215718
0.670761764800548
0.7060650155795243
0.7413682663585004
0.7766715171374766
0.8119747679164528
0.8472780186954291
0.8825812694744053
0.9178845202533815
0.9531877710323577
0.9884910218113339
1.0237942725903102
1.0590975233692863
1.0944007741482626
1.1297040249272388
1.1650072757062149
1.2003105264851912
1.2356137772641673
1.2709170280431437
1.3062202788221198
1.341523529601096
1.3768267803800722
1.4121300311590486
1.4474332819380247
1.4827365327170008
1.5180397834959771
1.5533430342749532
